## Import Library yang akan digunakan

In [1]:
from google_play_scraper import reviews
import pandas as pd

## Scraping Komentar pada Aplikasi Genshin Impact

In [4]:
# Scraping komentar dengan bintang 1-5
# Content, Score, At
_content = []
_score = []
_date = []

print("Memulai Scraping")
for i in [1,3,5]:
    results, _ = reviews(
        "com.finaccel.android",
        filter_score_with=i,
        count=4000,
        lang="id",
        country="id"
    )
    print("Append hasil kedalam list")
    for result in results:
        _content.append(result["content"])
        _score.append(result["score"])
        _date.append(result["at"])

Memulai Scraping
Append hasil kedalam list
Append hasil kedalam list
Append hasil kedalam list


In [5]:
# Membuat dataframe untuk hasil scraping
df_scraping = pd.DataFrame({"Review Date": _date, "Ratings" : _score, "Comment" : _content})
df_scraping["Ratings"].value_counts()

Ratings
1    4000
3    4000
5    4000
Name: count, dtype: int64

In [6]:
df_scraping.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Review Date  12000 non-null  datetime64[ns]
 1   Ratings      12000 non-null  int64         
 2   Comment      11999 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 281.4+ KB


In [7]:
df_scraping.to_csv("Dataset.csv", index=False)